<a href="https://colab.research.google.com/github/diogosantanaime/rec_proc/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalação da biblioteca 'datasets' usando o pip
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Importações dos pacotes

import random
import os

from datasets import load_dataset

 Carregar e preparar o PET (Process Extraction from Natural Language Text) Dataset.

 Fonte: https://huggingface.co/datasets/patriziobellan/PET

In [5]:
# Carregar os conjuntos de dados do repositório 'patriziobellan/PET'
token_classification = load_dataset("patriziobellan/PET", name='token-classification', split='test')
relations_extraction = load_dataset("patriziobellan/PET", name='relations-extraction', split='test')



 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]

1.0.1


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pet downloaded and prepared to /root/.cache/huggingface/datasets/patriziobellan___pet/token-classification/1.0.1/35e3f95d58d646f8cca1190108d17b8598f259d6ecaf1c995b1bf4f3adb1fc6f. Subsequent calls will reuse this data.


 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]

1.0.1


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pet downloaded and prepared to /root/.cache/huggingface/datasets/patriziobellan___pet/relations-extraction/1.0.1/35e3f95d58d646f8cca1190108d17b8598f259d6ecaf1c995b1bf4f3adb1fc6f. Subsequent calls will reuse this data.


In [6]:
# Criar uma lista de todas as frases
sentences = []

# Deixa um espaço em branco entre as sentenças
for sentence in token_classification["tokens"]:
    sentences += sentence + [' ']

# Obter todas as tags de entidade nomeada (NER) em uma única lista
ner = sum(relations_extraction["ner_tags"], [])


# Limpar as tags de entidade nomeada removendo espaços em branco
ner_tags = []

for tag in ner:
    tag_new = tag.replace(' ', '')
    ner_tags.append(tag_new)


# Criar uma lista de pares token e ner_tag (formato CoNLL)
conll = []
i = 0

for token in sentences:
    if token != ' ':
        conll.append(token + ' ' + ner_tags[i])
        i += 1
    else:
        conll.append(' ')
        i -= 1

# Dividir a lista CoNLL em frases individuais
sentences = []
sentence = []

for token in conll:
    if token == ' ':
        sentences.append(sentence)
        sentence = []
    else:
        sentence.append(token)

# Embaralhar as frases
random.shuffle(sentences)

# Definir as proporções de treinamento, teste e validação
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calcular o tamanho dos conjuntos de treinamento, desenvolvimento e teste
total_size = len(sentences)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = int(test_ratio * total_size)

# Dividir os dados em conjuntos de treinamento, desenvolvimento e teste
train_data = sentences[:train_size]
val_data = sentences[train_size:train_size+val_size]
test_data = sentences[train_size+val_size:train_size+val_size+test_size]

# Preparar os dados de treinamento no formato CoNLL
train_dataset = []

for sentence in train_data:
    train_dataset += sentence + ['\n']


In [7]:
len(sentences)

417

In [8]:
len(ner)

7569

Salvar o PET Dataset

In [ ]:
# Escrever os dados de treinamento em um arquivo CoNLL
with open('/content/drive/MyDrive/dataset/train_pet.txt', 'w') as f:
    for token in train_dataset:
        if token != '\n':
            f.write(token + '\n')
        else:
            f.write('\n')

# Preparar os dados de teste no formato CoNLL
test_dataset = []

for sentence in test_data:
    test_dataset += sentence + ['\n']

# Escrever os dados de teste em um arquivo CoNLL
with open('/content/drive/MyDrive/dataset/test_pet.txt', 'w') as f:
    for token in test_dataset:
        if token != '\n':
            f.write(token + '\n')
        else:
            f.write('\n')

# Preparar os dados de desenvolvimento no formato CoNLL
val_dataset = []

for sentence in val_data:
    val_dataset += sentence + ['\n']

# Escrever os dados de desenvolvimento em um arquivo CoNLL
with open('/content/drive/MyDrive/dataset/val_pet.txt', 'w') as f:
    for token in val_dataset:
        if token != '\n':
            f.write(token + '\n')
        else:
            f.write('\n')

Carregar e selecionar um subconjunto com 14000 amostras de treinamento, 3500 amostras de teste e 3500 amostras de validação do WikiNEuRal Dataset (en).

fonte: https://huggingface.co/datasets/Babelscape/wikineural

In [10]:
# Carrega os conjuntos de dados de treinamento, teste e validação
train_wikineural = load_dataset('Babelscape/wikineural', split='train_en')
test_wikineural = load_dataset('Babelscape/wikineural', split='test_en')
val_wikineural = load_dataset('Babelscape/wikineural', split='val_en')

# Rótulos das Entidades Nomeadas do dataset wikineural
tags_wikineural = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4,
                   'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


# Obtém o número total de linhas no conjunto de dados de treinamento
total_lines_train = len(train_wikineural)

# Define o número de linhas a serem selecionadas para o conjunto de treinamento
num_lines_to_select_train = 14000

# Cria uma lista vazia para armazenar o conjunto de treinamento selecionado
wikineural_train = []

# Seleciona aleatoriamente as linhas para o conjunto de treinamento
random_indices = random.sample(range(total_lines_train), num_lines_to_select_train)
for index in random_indices:
    wikineural_train.append(train_wikineural[index])

# Obtém o número total de linhas no conjunto de dados de teste
total_lines_test = len(test_wikineural)

# Define o número de linhas a serem selecionadas para o conjunto de teste
num_lines_to_select_test = 3500

# Cria uma lista vazia para armazenar o conjunto de teste selecionado
wikineural_test = []

# Seleciona aleatoriamente as linhas para o conjunto de teste
random_indices = random.sample(range(total_lines_test), num_lines_to_select_test)
for index in random_indices:
    wikineural_test.append(test_wikineural[index])

# Obtém o número total de linhas no conjunto de dados de validação
total_lines_val = len(val_wikineural)

# Define o número de linhas a serem selecionadas para o conjunto de validação
num_lines_to_select_val = 3500

# Cria uma lista vazia para armazenar o conjunto de validação selecionado
wikineural_val = []

# Seleciona aleatoriamente as linhas para o conjunto de validação
random_indices = random.sample(range(total_lines_val), num_lines_to_select_val)
for index in random_indices:
    wikineural_val.append(val_wikineural[index])

Extracting data files:   0%|          | 0/27 [00:00<?, ?it/s]

Generating test_de split: 0 examples [00:00, ? examples/s]

Generating test_en split: 0 examples [00:00, ? examples/s]

Generating test_es split: 0 examples [00:00, ? examples/s]

Generating test_fr split: 0 examples [00:00, ? examples/s]

...

Generating val_nl split: 0 examples [00:00, ? examples/s]

Generating val_pl split: 0 examples [00:00, ? examples/s]

Generating val_pt split: 0 examples [00:00, ? examples/s]

Generating val_ru split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Babelscape___parquet/Babelscape--wikineural-579d1dc98d2a6b93/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Carregar o CONLL 2003 Dataset

Fonte: https://huggingface.co/datasets/conll2003

In [30]:
train_conll03 = load_dataset('conll2003', split='train')
test_conll03 = load_dataset('conll2003', split='test')
val_conll03 = load_dataset('conll2003', split='validation')

tags_conll03 = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4,
                'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

Função que cria um arquivo no formato CONLL.

In [ ]:
def create_conll_file(dataset, filename, tags):
    # Cria o diretório para armazenar o arquivo CoNLL
    directory = '/content/drive/MyDrive/dataset'
    os.makedirs(directory, exist_ok=True)

    # Define o caminho do arquivo CoNLL
    filepath = os.path.join(directory, filename)

    # Abre o arquivo CoNLL no modo de escrita
    with open(filepath, 'w', encoding='utf-8') as file:
        # Itera sobre os dados do conjunto de dados
        for data in dataset:
            tokens = data['tokens']
            ner_tags = data['ner_tags']

            # Escreve cada token e rótulo de NER no arquivo
            for token, tag in zip(tokens, ner_tags):
                file.write(token + '\t')

                # Mapeia o rótulo de NER para o valor correspondente
                if tag in tags.values():
                    tag_mapped = list(tags.keys())[list(tags.values()).index(tag)]
                else:
                    tag_mapped = 'O'

                file.write(tag_mapped + '\n')

            file.write('\n')

Salvar o WikiNEuRal Dataset (en)

In [ ]:
create_conll_file(wikineural_train, 'train_wikineural.txt', tags_wikineural)
create_conll_file(wikineural_test, 'test_wikineural.txt', tags_wikineural)
create_conll_file(wikineural_val, 'val_wikineural.txt', tags_wikineural)



Salvar o Conll03 Dataset

In [ ]:
create_conll_file(conll03_train, 'train_conll03.txt', tags_conll03)
create_conll_file(conll03_test, 'test_conll03.txt', tags_conll03)
create_conll_file(conll03_val, 'val_conll03.txt', tags_conll03)